In [20]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import requests
from datetime import datetime

load_dotenv()
dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]
erAppId = os.environ["APP_ID"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][1]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order',
    'search_read',
    [],
    {'fields': [
        "id",
        "name",
        "is_subscription",
        "create_date",
        "order_line",
        "order_log_ids"
    ]}
)



#id_field_into_1value(df)

df = pd.DataFrame(df)
print(df)

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
#df.to_sql('odoo_subscriptions_test', con=engine, if_exists='replace', index=False)

assoc_results = []
itterator = df.copy()
for result in itterator:
    try:
        lists_to_append = result["associations"]["deals"]["results"]
        to_append = []
        for i in lists_to_append:
            dict_to_append = {}
            dict_to_append["quote_id"] = result["id"]
            dict_to_append["deal_id"] = i["id"]
            to_append.append(dict_to_append)
        assoc_results.append(to_append)
    except:
        assoc_results.append([{"quote_id": result["id"], "deal_id": None}])
contacts_id = []
deal_id = []
for index, item in enumerate(assoc_results):
    i = 0
    for a, b in item:
        contacts_id.append(item[i][a])
        deal_id.append(item[i][b])
        i += 1
df = pd.DataFrame({"quote_id": contacts_id, "deal_id": deal_id})




       id    name  is_subscription          create_date  \
0    2564  S01687             True  2024-03-07 04:59:23   
1    2563  S01686             True  2024-03-05 15:38:04   
2    2562  S01685             True  2024-03-04 18:54:38   
3    2561  S01684             True  2024-03-04 18:31:55   
4    2560  S01683             True  2024-02-23 16:30:40   
..    ...     ...              ...                  ...   
584     6  S00006             True  2023-04-24 20:21:14   
585     5  S00005             True  2023-04-24 20:02:48   
586     4  S00004             True  2023-04-24 19:26:28   
587     3  S00003             True  2023-04-24 18:55:14   
588   403  S00404             True  2023-07-26 15:52:55   

                                            order_line  \
0                                               [1512]   
1                                   [1508, 1509, 1510]   
2                                         [1504, 1505]   
3                                               [1503]   
4

589